In [ ]:
%pip install dash plotly pandas


In [ ]:
!pip list


In [33]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import numpy as np
import plotly.express as px

# Sample data generation
np.random.seed(42)
date_range = pd.date_range(start='2024-01-01', end=pd.Timestamp.now(), freq='D')
data = {
    'Date': date_range,
    'TimeSpent': np.random.normal(loc=10, scale=2, size=len(date_range)).round(2),
    'AvgTimePerVisit': np.random.normal(loc=5, scale=1, size=len(date_range)).round(2),
    'Day1CancelRate': np.random.normal(loc=0.1, scale=0.02, size=len(date_range)).round(4) * 100,
    'Week1CancelRate': np.random.normal(loc=0.15, scale=0.03, size=len(date_range)).round(4) * 100,
    'Week2CancelRate': np.random.normal(loc=0.2, scale=0.04, size=len(date_range)).round(4) * 100,
    'RC1Rate': np.random.normal(loc=0.58, scale=0.01, size=len(date_range)).round(4) * 100,
    'DailyReturnUsers': np.random.randint(50, 200, size=len(date_range)),
    'DocumentsCreated': np.random.randint(20, 100, size=len(date_range)),
    'Logins': np.random.randint(100, 400, size=len(date_range)),
    'Country': np.random.choice(['US', 'UK', 'CA', 'AU', 'DE', 'FR', 'IN', 'BR', 'JP', 'MX', 'Other'], len(date_range)),
    'Device': np.random.choice(['Desktop', 'Mobile'], len(date_range)),
    'Medium': np.random.choice(['SEM', 'SEO', 'Display', 'CPC', 'Affiliate', 'Other'], len(date_range)),
    'Source': np.random.choice(['Google', 'Bing', 'Direct', 'Organic', 'Other'], len(date_range)),
    'Jobs': np.random.randint(0, 2, size=len(date_range)),
    'ResumeReview': np.random.randint(0, 2, size=len(date_range)),
    'ResumeCheck': np.random.randint(0, 2, size=len(date_range)),
    'CoverLetterCreation': np.random.randint(0, 2, size=len(date_range)),
    'FeedbackTool': np.random.randint(0, 2, size=len(date_range)),
    'ResumeEdit': np.random.randint(0, 2, size=len(date_range)),
    'AdditionalResumeCreated': np.random.randint(0, 2, size=len(date_range)),
}
df = pd.DataFrame(data)

# Resample data to weekly
df_weekly = df.resample('W-Mon', on='Date').mean().reset_index()

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1('MPR Retention Dashboard | Not Real Data | For demo', style={'fontSize': '32px', 'textAlign': 'center', 'color': 'teal'}),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'Time Spent on Retention Pages (minutes)', 'value': 'TimeSpent'},
            {'label': 'Average Time per Visit (minutes)', 'value': 'AvgTimePerVisit'},
            {'label': 'Daily Return Users', 'value': 'DailyReturnUsers'},
            {'label': 'Documents Created', 'value': 'DocumentsCreated'},
            {'label': 'Logins', 'value': 'Logins'}
        ],
        value='TimeSpent',
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Graph(id='line-chart'),
    html.Div([
        html.Div([
            html.Label('Filter by Country'),
            dcc.Dropdown(
                id='country-filter',
                options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': country, 'value': country} for country in df['Country'].unique() if country != 'CN'],
                multi=True,
                value=['ALL'],
                placeholder="Country"
            )
        ], style={'flex': '1', 'padding': '10px', 'minWidth': '200px'}),
        html.Div([
            html.Label('Filter by Device'),
            dcc.Dropdown(
                id='device-filter',
                options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': device, 'value': device} for device in df['Device'].unique()],
                multi=True,
                value=['ALL'],
                placeholder="Device"
            )
        ], style={'flex': '1', 'padding': '10px', 'minWidth': '200px'}),
        html.Div([
            html.Label('Filter by Medium'),
            dcc.Dropdown(
                id='medium-filter',
                options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': medium, 'value': medium} for medium in df['Medium'].unique()],
                multi=True,
                value=['ALL'],
                placeholder="Medium"
            )
        ], style={'flex': '1', 'padding': '10px', 'minWidth': '200px'}),
        html.Div([
            html.Label('Filter by Source'),
            dcc.Dropdown(
                id='source-filter',
                options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': source, 'value': source} for source in df['Source'].unique()],
                multi=True,
                value=['ALL'],
                placeholder="Source"
            )
        ], style={'flex': '1', 'padding': '10px', 'minWidth': '200px'})
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-around'}),
    html.H2('Cancel Rates/ Retention Rates', style={'fontSize': '28px', 'textAlign': 'center', 'color': 'darkgreen', 'marginTop': '20px'}),
    dcc.Dropdown(
        id='cancel-rate-metric-dropdown',
        options=[
            {'label': 'Day 1 Cancel Rate', 'value': 'Day1CancelRate'},
            {'label': 'Week 1 Cancel Rate', 'value': 'Week1CancelRate'},
            {'label': 'Week 2 Cancel Rate', 'value': 'Week2CancelRate'},
            {'label': 'RC1 Rate', 'value': 'RC1Rate'}
        ],
        value='Day1CancelRate',
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Graph(id='cancel-rate-chart'),
    html.H2('Engagement Rate by Feature', style={'fontSize': '28px', 'textAlign': 'center', 'color': 'darkgreen', 'marginTop': '20px'}),
    dcc.DatePickerRange(
        id='engagement-date-picker',
        start_date='2024-01-01',
        end_date=pd.Timestamp.now().date()
    ),
    dcc.Graph(id='engagement-rate-chart')
])

@app.callback(
    Output('line-chart', 'figure'),
    Input('metric-dropdown', 'value'),
    Input('country-filter', 'value'),
    Input('device-filter', 'value'),
    Input('medium-filter', 'value'),
    Input('source-filter', 'value')
)
def update_chart(selected_metric, selected_countries, selected_devices, selected_mediums, selected_sources):
    filtered_df = df_weekly.copy()
    
    if 'ALL' not in selected_countries:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_countries)]
    if 'ALL' not in selected_devices:
        filtered_df = filtered_df[filtered_df['Device'].isin(selected_devices)]
    if 'ALL' not in selected_mediums:
        filtered_df = filtered_df[filtered_df['Medium'].isin(selected_mediums)]
    if 'ALL' not in selected_sources:
        filtered_df = filtered_df[filtered_df['Source'].isin(selected_sources)]
    
    fig = px.line(filtered_df, x='Date', y=selected_metric, title=selected_metric)
    yaxis_title = f'{selected_metric} (minutes)' if 'Time' in selected_metric else selected_metric
    fig.update_layout(title={'font_size': 24}, xaxis_title='Date (Weekly)', yaxis_title=yaxis_title)
    return fig

@app.callback(
    Output('cancel-rate-chart', 'figure'),
    Input('cancel-rate-metric-dropdown', 'value'),
    Input('country-filter', 'value'),
    Input('device-filter', 'value'),
    Input('medium-filter', 'value'),
    Input('source-filter', 'value')
)
def update_cancel_rate_chart(selected_metric, selected_countries, selected_devices, selected_mediums, selected_sources):
    filtered_df = df_weekly.copy()
    
    if 'ALL' not in selected_countries:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_countries)]
    if 'ALL' not in selected_devices:
        filtered_df = filtered_df[filtered_df['Device'].isin(selected_devices)]
    if 'ALL' not in selected_mediums:
        filtered_df = filtered_df[filtered_df['Medium'].isin(selected_mediums)]
    if 'ALL' not in selected_sources:
        filtered_df = filtered_df[filtered_df['Source'].isin(selected_sources)]
    
    fig = px.line(filtered_df, x='Date', y=selected_metric, title=selected_metric)
    yaxis_title = f'{selected_metric} (%)'
    fig.update_layout(title={'font_size': 24}, xaxis_title='Date (Weekly)', yaxis_title=yaxis_title)
    if selected_metric == 'RC1Rate':
        fig.update_yaxes(range=[50, 58])
    return fig

@app.callback(
    Output('engagement-rate-chart', 'figure'),
    Input('engagement-date-picker', 'start_date'),
    Input('engagement-date-picker', 'end_date')
)
def update_engagement_chart(start_date, end_date):
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    engagement_data = filtered_df[['Jobs', 'ResumeReview', 'ResumeCheck', 'CoverLetterCreation', 'FeedbackTool', 'ResumeEdit', 'AdditionalResumeCreated']].mean() * 100
    engagement_df = pd.DataFrame(engagement_data, columns=['EngagementRate']).reset_index().rename(columns={'index': 'Feature'})
    
    fig = px.bar(engagement_df, x='Feature', y='EngagementRate', title='Engagement Rate by Feature (within 14 days of subscription)',orientation='v')
    fig.update_layout(title={'font_size': 24}, xaxis_title='Feature', yaxis_title='Engagement Rate (%)')
    return fig

# Run app
if __name__ == '__main__':
    app.run_server(debug=True)

